# Neural Style Transfer with Keras

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

---
## Introduction

General process 
1. Set up a network that computes layer-activations for the style-reference image, the target image, and the generated image at the same time. 
2. Use these layer activations to define the loss function.
3. Set up a gradient-ascnt process to minimize this loss function by modifying the generated image. 

### Defining Initial Variables

In [0]:
from keras.preprocessing.image import load_img, img_to_array

# Image Paths 
target_image_path = '/content/drive/My Drive/Colab Notebooks/inputs/IU_portrait.png'
style_reference_image_path = '/content/drive/My Drive/Colab Notebooks/style/the-starry-night.jpg'

# Dimensins for the generated picture
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

### Auxiliary Functions 

In [0]:
import numpy as np
from keras.applications import vgg19 

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img 

def deprocess_image(image):
    # Reverses a transformation in vgg19.preprocess_input()
    image[:, :, 0] += 103.939
    image[:, :, 1] += 116.779
    image[:, :, 2] += 123.68 
    # Convert from BGR to RGB (also vgg19.preprocess_input())
    image = image[:, :, ::-1]
    return np.clip(image, 0, 255).astype('uint8')

### Loading VGG19 and Applying to Three Images

In [0]:
from keras import backend as K 

# Generated image is a placeholder others are constant 
target_image = K.constant(preprocess_image(target_image_path))
style_image = K.constant(preprocess_image(style_reference_image_path))
generated_image = K.placeholder((1, img_height, img_width, 3))

# Combine the images into a "batch" 
input_tensor = K.concatenate([target_image, style_image, generated_image], axis=0)

# Load the model 
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

### Defining the Loss Function(s)

In [0]:
def content_loss(target, generated):
    return K.sum(K.square(generated - target))

def gram_matrix(x):
    """Captures the 'style' of an image."""
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram 

def style_loss(style, generated):
    S = gram_matrix(style)
    G = gram_matrix(generated)
    channels = 3 
    size = img_height * img_width
    return K.sum(K.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width - 1, :] -
                 x[:, 1:, :img_width - 1, :])
    b = K.square(x[:, :img_height - 1, :img_width - 1, :] -
                 x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

### Defining the Final Loss Function 

In [0]:
layers_dict = {layer.name: layer.output for layer in model.layers}
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

# Weights in the weighted average 
total_variation_weight = 1e-2 
style_weight = 0.5 
content_weight = 0.5

# Overall Loss 
loss = K.variable(0.)

# Loss += Content Loss 
layer_features = layers_dict[content_layer]
target_image_featues = layer_features[0, :, :, :]
generated_image_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_featues,
                                            generated_image_features)

# Loss += Style Loss 
for layer_name in style_layers: 
    layer_features = layers_dict[layer_name]
    style_image_features = layer_features[1, :, :, :]
    generated_image_features = layer_features[2, :, :, :]
    temp_style_loss = style_loss(style_image_features, 
                                 generated_image_features)
    loss = loss + (style_weight / len(style_layers)) * temp_style_loss
    
# Loss += Variation Loss 
loss = loss + total_variation_weight * total_variation_loss(generated_image)

### Setting up Gradient-Descent 

In [0]:
grads = K.gradients(loss, generated_image)[0]
fetch_loss_and_grads = K.function([generated_image], [loss, grads])

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

### Style Transfer Loop 

In [55]:
from scipy.optimize import fmin_l_bfgs_b
import imageio

base_path = '/content/drive/My Drive/Colab Notebooks/outputs/'
result_prefix = 'result'
iterations = 30

x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(1, iterations + 1):
    print('Start of iteration', i)
    x, min_val, _ = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = base_path + str(i) + '.png'
    imageio.imwrite(fname, img)

final_path = base_path + 'final.png'
imageio.imwrite(final_path, img)

Start of iteration 1
Current loss value: 1192790300.0
Start of iteration 2
Current loss value: 576547700.0
Start of iteration 3
Current loss value: 413295300.0
Start of iteration 4
Current loss value: 337355140.0
Start of iteration 5
Current loss value: 294651900.0
Start of iteration 6
Current loss value: 264794980.0
Start of iteration 7
Current loss value: 238312660.0
Start of iteration 8
Current loss value: 222517490.0
Start of iteration 9
Current loss value: 208912460.0
Start of iteration 10
Current loss value: 193770380.0
Start of iteration 11
Current loss value: 184535520.0
Start of iteration 12
Current loss value: 178183970.0
Start of iteration 13
Current loss value: 171971380.0
Start of iteration 14
Current loss value: 166173570.0
Start of iteration 15
Current loss value: 162028960.0
Start of iteration 16
Current loss value: 158924140.0
Start of iteration 17
Current loss value: 155823730.0
Start of iteration 18
Current loss value: 153069360.0
Start of iteration 19
Current loss v

---
## Creating GIFs from Outputs 
Before showing you some of my results, here is a helper function you can use to transform a directory of numerically ordered photos (1.png, 2.png, 3.jpg, etc.) into a GIF. This allows us to see the photo transforming over time, which is pretty cool.

In [37]:
import os 
import re 
import imageio

# only keep files that followed the `number.png` convention
file_format = re.compile(r'(\d+).png')

# Sort the images numerically (e.g. 1.png, 2.png, ...)
def sort_key(filename):
    """Return the numeric portion of filename."""
    return int(filename.split('.')[0])

# Transform the files in the directory into a gif 
def images_to_gif(source_dir, name='final.gif'):
    """Turn a directory of numerically labeled photos into gifs."""
    files = sorted([fname for fname in os.listdir(source_dir) 
                    if file_format.match(fname)], key=sort_key)
    images = [imageio.imread(os.path.join(source_dir, fname)) 
              for fname in files]
    imageio.mimsave(os.path.join(source_dir, name), images, 
                    'GIF', duration=0.05)

I used this to generate an image for all of my photos.

In [38]:
source = '/Users/jtbergman/GitHub/deep-learning-code/cnn06/outputs/img5/'
images_to_gif(source)

---
## Results
_Note: Links to all of the files are provided at the end of the notebook._

### Example 1
```python 
# Images
content_image = './inputs/germany.jpg'
style_image = './style/the_shipwreck_of_the_minotaur.jpg'

# Parameters
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
```

![Neckarfront in Tübingeng, Germany](./outputs/img1/final.gif)

### Example 2
```python 
# Images
content_image = './inputs/IU_portrait.png'
style_image = './style/femme_nue_assise_pablo_picasso.jpg'

# Parameters
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025
```

![IU + Femme Nue Assise](./outputs/img2/final.gif)

### Example 3
```python 
# Images
content_image = './inputs/IU_portrait.png'
style_image = './style/composition_vii.jpg'

# Parameters
total_variation_weight = 1e-2
style_weight = 0.5
content_weight = 0.5
```

![IU + composition vii](./outputs/img3/final.gif)

### Example 4
```python 
# Images
content_image = './inputs/IU_portrait.png'
style_image = './style/the-starry-night.jpg'

# Parameters
total_variation_weight = 1e-2
style_weight = 0.5
content_weight = 0.5
```

![IU + Starry Night](./outputs/img4/final.gif)

---
## Summary 
In my opinion, this is a lot cooler than DeepDream because we have way more influence over what the final image look likes. I actually think it would be better to actual slow the learning down so that we can create GIFs with much smoother transitions. If you look a the outputs,by just the third iteration the generated image is usually a lot closer to the final output than the content image. 

### Original Neural Style Transfer Paper
You can find the original neural style transfer paper [here](https://arxiv.org/pdf/1508.06576.pdf)

### Content Images 
The first content image is of the Neckarfront in Tübingen, Germany. If you look at the original paper this is one of the images they used. The second content image is a portrait of IU – a Korean singer-songwriter and actress. If you're interested... [IU - 삐삐](https://youtu.be/nM0xDI5R50E).

### Style Images 
All of the style images were used in the original NST paper as well. 
+ _The Shipwreck of the Minotaur_ by JMW Turner (1805)
+ _Femme nue assise_ by Pablo Picasso (1910)
+ _Composition VII_ by Wassily Kandinsky (1913)
+ _The Starry Night_ by Vincent van Gogh (1889)